In [1]:
import pandas as pd

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error

def read_dataframe(path):
    df = pd.read_parquet(path)
    df["duration"] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime).dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    df[["PULocationID", "DOLocationID"]] = df[["PULocationID", "DOLocationID"]].astype(str)
    return df

df_train = read_dataframe("C:\\Users\\Daniela de la Torre\\OneDrive\\Escritorio\\Universidad\\5to_Semestre\\ProyectoCD\\nyc-taxi-predictions-2025-1\\data\\green_tripdata_2025-01.parquet")
df_val = read_dataframe("C:\\Users\\Daniela de la Torre\\OneDrive\\Escritorio\\Universidad\\5to_Semestre\\ProyectoCD\\nyc-taxi-predictions-2025-1\\data\\green_tripdata_2025-02.parquet")

df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

categorical = ["PU_DO"]
numerical = ["trip_distance"]

dv = DictVectorizer()
X_train = dv.fit_transform(df_train[categorical + numerical].to_dict(orient="records"))
X_val = dv.transform(df_val[categorical + numerical].to_dict(orient="records"))

y_train = df_train["duration"].values
y_val = df_val["duration"].values

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("class-nyc-taxi-experiment")

with mlflow.start_run(run_name="lasso_alpha_0.1"):
    alpha = 0.1
    mlflow.log_param("alpha", alpha)

    model = Lasso(alpha=alpha)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)

    mlflow.log_metric("rmse", rmse)
    mlflow.sklearn.log_model(model, "model")

    print(f"✅ Run finalizado. RMSE = {rmse:.4f}")

2025/10/22 11:39:03 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/10/22 11:39:03 INFO mlflow.store.db.utils: Updating database tables
2025-10-22 11:39:03 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-10-22 11:39:03 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
2025-10-22 11:39:03 INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
2025-10-22 11:39:03 INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
2025-10-22 11:39:03 INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
2025-10-22 11:39:03 INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
2025-10-22 11:39:03 INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
2025-10-22 11:39:03 INFO  [alembic.runtime.mig

✅ Run finalizado. RMSE = 8.9926


In [5]:
from sklearn.ensemble import GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

In [6]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_name="nyc-taxi-model-registry-example")

2025/10/22 11:39:36 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-model-registry-example' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///c:/Users/Daniela de la '
 'Torre/OneDrive/Escritorio/Universidad/5to_Semestre/ProyectoCD/nyc-taxi-predictions-2025-1/notebooks/mlruns/2'), creation_time=1761154776820, experiment_id='2', last_update_time=1761154776820, lifecycle_stage='active', name='nyc-taxi-model-registry-example', tags={}>

In [7]:
mlflow.sklearn.autolog()

In [8]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [9]:
models = [
    
    {"model": GradientBoostingRegressor,
     "params": {"n_estimators": 100, "learning_rate": 0.3, "max_depth": 25, "random_state": 42},
     },
    
    {"model": ExtraTreesRegressor,
     "params": {"n_estimators": 100, "max_depth": 15, "random_state": 42},
     },
    
    {"model": LinearSVR,
     "params": {"C": 1.0, "epsilon": 0}, 
     },

]

In [14]:
with mlflow.start_run(run_name="Nested Runs"):
    for model in models:
        
        model_class = model["model"]
        model_name = model_class.__name__
        params = model["params"]
        
        with mlflow.start_run(run_name=model_name,nested=True):
            
            ml_model = model_class(**params)
           
            ml_model.fit(X_train, y_train)
    
            y_pred = ml_model.predict(X_val)
            
            rmse = root_mean_squared_error(y_val, y_pred)
            mlflow.log_metric("rmse", rmse)
            
            !mkdir models
            with open("models/preprocessor.b", "wb") as f_out:
                pickle.dump(dv, f_out)
                
            mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025/10/22 11:53:41 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.
2025/10/22 11:54:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.
c:\Users\Daniela de la Torre\OneDrive\Escritorio\Universidad\5to_Semestre\ProyectoCD\nyc-taxi-predictions-2025-1\.venv\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
2025/10/22 11:54:11 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Ya existe el subdirectorio o el archivo models.


In [16]:
from sklearn.ensemble import RandomForestRegressor


with mlflow.start_run(run_name="RandomForestRegressor"):
    ml_model = RandomForestRegressor(
        n_estimators=100,
        max_depth=15,
        random_state=42
    )
    
    ml_model.fit(X_train, y_train)
    
    mlflow.sklearn.log_model(
        sk_model=model, 
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )
    
    y_pred = ml_model.predict(X_val)
    
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    
    # !mkdir models
    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

2025-10-22 11:58:16 INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
2025-10-22 11:58:16 INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'nyc-taxi-model'.
Created version '1' of model 'nyc-taxi-model'.


In [17]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="nyc-taxi-model"
)

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...


MlflowException: Run with id=nyc-taxi-model not found

In [18]:
from mlflow import MlflowClient

client = MlflowClient(tracking_uri="sqlite:///mlflow.db")

In [19]:
client.create_registered_model(name="nyc-taxi-model")

MlflowException: Registered Model (name=nyc-taxi-model) already exists.

In [20]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = client.create_model_version(
    name="nyc-taxi-model",
    source=run_uri,
    run_id=run_id
)

In [28]:
# Ver todas las versiones del modelo
client = MlflowClient()
versions = client.search_model_versions(f"name='nyc-taxi-model'")

print("Versiones disponibles:")
for version in versions:
    print(f"Version: {version.version}, Stage: {version.current_stage}")

Versiones disponibles:
Version: 2, Stage: None
Version: 1, Stage: None


In [29]:
# Champion
client.set_registered_model_alias(
    name="nyc-taxi-model",
    alias="champion",
    version=1
)

In [30]:
# set the "challenger" alias to version 2
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger",
    version=2
)

In [31]:
client.update_model_version(
    name="nyc-taxi-model",
    version=1,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1761155896696, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1761156966406, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='ec6883580a184e38b6a720e9b2c5596a', run_link=None, source='models:/m-d93f97c2914b4d1aa778038268a049bd', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [32]:
# get a model version by alias
client.get_model_version_by_alias(
    name="nyc-taxi-model",
    alias="champion"
)

<ModelVersion: aliases=['champion'], creation_timestamp=1761155896696, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1761156966406, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='ec6883580a184e38b6a720e9b2c5596a', run_link=None, source='models:/m-d93f97c2914b4d1aa778038268a049bd', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [33]:
# delete the alias
client.delete_registered_model_alias(
    name="nyc-taxi-model", 
    alias="challenger"
)

In [34]:
client.update_model_version(
    name="nyc-taxi-model",
    version=1,
    description="This model version is a scikit-learn random forest containing 100 decision trees",
)

<ModelVersion: aliases=['champion'], creation_timestamp=1761155896696, current_stage='None', deployment_job_state=None, description=('This model version is a scikit-learn random forest containing 100 decision '
 'trees'), last_updated_timestamp=1761157017307, metrics=None, model_id=None, name='nyc-taxi-model', params=None, run_id='ec6883580a184e38b6a720e9b2c5596a', run_link=None, source='models:/m-d93f97c2914b4d1aa778038268a049bd', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [36]:
import mlflow.sklearn

with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=ml_model,  # tu modelo entrenado
        artifact_path="model", 
        registered_model_name="nyc-taxi-model"
    )


Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-model'.


In [40]:
import mlflow.sklearn

with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=ml_model,          # tu modelo entrenado
        artifact_path="model",       # carpeta donde se guarda
        registered_model_name="nyc-taxi-model"
    )


Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-model'.


In [46]:
import mlflow.sklearn

with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=ml_model,           # tu modelo entrenado
        artifact_path="model",
        registered_model_name="nyc-taxi-model"  # registrarlo
    )


Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '6' of model 'nyc-taxi-model'.


In [49]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
model_version = 1

model_uri = f"models:/{model_name}/{model_version}"

model = mlflow.pyfunc.load_model(
    model_uri=model_uri,
)

model.predict(X_val)

MlflowException: Model does not have the "python_function" flavor

In [48]:
import mlflow.sklearn
import mlflow.pyfunc

# Registrar un nuevo modelo como pyfunc
with mlflow.start_run():
    mlflow.sklearn.log_model(
        sk_model=ml_model,
        artifact_path="model",
        registered_model_name="nyc-taxi-model"
    )

# Recuperar la última versión registrada
client = mlflow.tracking.MlflowClient()
latest_version = client.get_latest_versions("nyc-taxi-model", stages=["None"])[0].version

model_uri = f"models:/nyc-taxi-model/{latest_version}"
model = mlflow.pyfunc.load_model(model_uri)
preds = model.predict(X_val)


Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
Created version '7' of model 'nyc-taxi-model'.


In [50]:
import mlflow.pyfunc

model_name = "nyc-taxi-model"
alias = "champion"

model_uri = f"models:/{model_name}@{alias}"

champion_version = mlflow.pyfunc.load_model(
    model_uri=model_uri
)

champion_version.predict(X_val)

MlflowException: Model does not have the "python_function" flavor

In [51]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, alias, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}@{alias}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [52]:
df = read_dataframe("C:\\Users\\Daniela de la Torre\\OneDrive\\Escritorio\\Universidad\\5to_Semestre\\ProyectoCD\\nyc-taxi-predictions-2025-1\\data\\green_tripdata_2025-03.parquet")

In [55]:
run_id = input("Ingrese el run_id")

client.download_artifacts(
    run_id=run_id, 
    path='preprocessor', 
    dst_path='.'
)

MlflowException: Run with id=nyc-taxi-predictions-2025 not found

In [56]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

FileNotFoundError: [Errno 2] No such file or directory: 'preprocessor/preprocessor.b'

In [57]:
X_test = preprocess(df, dv)

In [58]:
target = "duration"
y_test = df[target].values

In [59]:
%time test_model(name="nyc-taxi-model", alias="champion", X_test=X_test, y_test=y_test)

CPU times: total: 0 ns
Wall time: 0 ns


NameError: name 'champion' is not defined

In [60]:
%time test_model(name=model_name, alias="challenger", X_test=X_test, y_test=y_test)

CPU times: total: 0 ns
Wall time: 0 ns


NameError: name 'challenger' is not defined

In [61]:
client.set_registered_model_alias(
    name="nyc-taxi-model", 
    alias="champion",
    version=100
)

MlflowException: Model Version (name=nyc-taxi-model, version=100) not found